# Data Science Final Project
Charles Laurent, Jenny Park, AJ Marino

## Part 3 - Adding external data

### Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import metrics

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

pd.set_option('display.max_columns', None)

In [3]:
# TODO - Update path to files
datadir='../dev/project/CollegeScorecard_Raw_Data'

# TODO set features_dir 
features_dir = './data'


In [4]:
# import collegedatalib
import collegedatahelper

### Import Data (formated through library)

In [5]:
%%time
# Import data (add state id)
data_dict = collegedatahelper.create_data_dict(datadir, features_dir, ['STABBR'])

CPU times: user 55.6 s, sys: 7.88 s, total: 1min 3s
Wall time: 55.3 s


In [6]:
df = data_dict['2012_13']
df_13 = data_dict['2013_14']
df_new = data_dict['2017_18']

In [7]:
# CONTROL 
# 1 - Public
# 2 - Private non-profit
# 3 - Private for-profit
df_pub = df_13['public']
df_priv = df_13['private']
df_priv_profit = df_13['private_for_profit']

In [8]:
for key in data_dict:
    print(key)
    print(data_dict[key]['public'].shape)
    print(data_dict[key]['private'].shape)
    print(data_dict[key]['private_for_profit'].shape)

2012_13
(1662, 417)
(1411, 417)
(2759, 417)
2013_14
(1673, 417)
(1428, 417)
(2801, 417)
2014_15
(1669, 417)
(1439, 417)
(2780, 417)
2015_16
(2063, 1977)
(2007, 1977)
(3523, 1977)
2016_17
(2064, 1977)
(2009, 1977)
(3102, 1977)
2017_18
(2076, 1977)
(2041, 1977)
(2995, 1977)


In [9]:
df_pub['STABBR'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'GU', 'MP', 'PR', 'VI'],
      dtype=object)

# Add State income data

In [10]:
state_abb_df = pd.read_csv(f'{features_dir}/state_abbreviations.csv', low_memory=False)[['State', 'Code']]

In [11]:
state_abb_df.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [12]:
# income_df = pd.read_csv(f'{features_dir}/state_income/ACS_10_1YR_R1901.US01PRF.csv', low_memory=False)
years = collegedatahelper.years_train
years.extend(collegedatahelper.years_test)

In [13]:
income_dictionary = {}
for year in years:
    partial_year = year[2:4]
    income_df = pd.read_csv(f'{features_dir}/state_income/ACS_' + partial_year + '_1YR_R1901.US01PRF.csv', low_memory=False)[['GRT_STUB.display-label', 'EST']]
    
    income_df = income_df.merge(state_abb_df, left_on='GRT_STUB.display-label', right_on='State')[['Code', 'EST']]
    income_dictionary[year] = income_df

    

In [14]:
income_dictionary['2012_13'].head()

,Code,EST
0,MD,71122
1,NJ,69667
2,AK,67712
3,CT,67276
4,DC,66583


In [15]:
income_dictionary['2013_14'].head()

,Code,EST
0,MD,72483
1,AK,72237
2,NJ,70165
3,HI,68020
4,DC,67572


In [16]:
df_pub.head()

,STABBR,CONTROL,ADM_RATE,ADM_RATE_ALL,SATVR25,SATVR75,SATMT25,SATMT75,SATWR25,SATWR75,SATVRMID,SATMTMID,SATWRMID,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTWR25,ACTWR75,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,PCIP15,PCIP16,PCIP19,PCIP22,PCIP23,PCIP24,PCIP25,PCIP26,PCIP27,PCIP29,PCIP30,PCIP31,PCIP38,PCIP39,PCIP40,PCIP41,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP01CERT1,CIP01CERT2,CIP01ASSOC,CIP01CERT4,CIP01BACHL,CIP03CERT1,CIP03CERT2,CIP03ASSOC,CIP03CERT4,CIP03BACHL,CIP04CERT1,CIP04CERT2,CIP04ASSOC,CIP04CERT4,CIP04BACHL,CIP05CERT1,CIP05CERT2,CIP05ASSOC,CIP05CERT4,CIP05BACHL,CIP09CERT1,CIP09CERT2,CIP09ASSOC,CIP09CERT4,CIP09BACHL,CIP10CERT1,CIP10CERT2,CIP10ASSOC,CIP10CERT4,CIP10BACHL,CIP11CERT1,CIP11CERT2,CIP11ASSOC,CIP11CERT4,CIP11BACHL,CIP12CERT1,CIP12CERT2,CIP12ASSOC,CIP12CERT4,CIP12BACHL,CIP13CERT1,CIP13CERT2,CIP13ASSOC,CIP13CERT4,CIP13BACHL,CIP14CERT1,CIP14CERT2,CIP14ASSOC,CIP14CERT4,CIP14BACHL,CIP15CERT1,CIP15CERT2,CIP15ASSOC,CIP15CERT4,CIP15BACHL,CIP16CERT1,CIP16CERT2,CIP16ASSOC,CIP16CERT4,CIP16BACHL,CIP19CERT1,CIP19CERT2,CIP19ASSOC,CIP19CERT4,CIP19BACHL,CIP22CERT1,CIP22CERT2,CIP22ASSOC,CIP22CERT4,CIP22BACHL,CIP23CERT1,CIP23CERT2,CIP23ASSOC,CIP23CERT4,CIP23BACHL,CIP24CERT1,CIP24CERT2,CIP24ASSOC,CIP24CERT4,CIP24BACHL,CIP25CERT1,CIP25CERT2,CIP25ASSOC,CIP25CERT4,CIP25BACHL,CIP26CERT1,CIP26CERT2,CIP26ASSOC,CIP26CERT4,CIP26BACHL,CIP27CERT1,CIP27CERT2,CIP27ASSOC,CIP27CERT4,CIP27BACHL,CIP29CERT1,CIP29CERT2,CIP29ASSOC,CIP29CERT4,CIP29BACHL,CIP30CERT1,CIP30CERT2,CIP30ASSOC,CIP30CERT4,CIP30BACHL,CIP31CERT1,CIP31CERT2,CIP31ASSOC,CIP31CERT4,CIP31BACHL,CIP38CERT1,CIP38CERT2,CIP38ASSOC,CIP38CERT4,CIP38BACHL,CIP39CERT1,CIP39CERT2,CIP39ASSOC,CIP39CERT4,CIP39BACHL,CIP40CERT1,CIP40CERT2,CIP40ASSOC,CIP40CERT4,CIP40BACHL,CIP41CERT1,CIP41CERT2,CIP41ASSOC,CIP41CERT4,CIP41BACHL,CIP42CERT1,CIP42CERT2,CIP42ASSOC,CIP42CERT4,CIP42BACHL,CIP43CERT1,CIP43CERT2,CIP43ASSOC,CIP43CERT4,CIP43BACHL,CIP44CERT1,CIP44CERT2,CIP44ASSOC,CIP44CERT4,CIP44BACHL,CIP45CERT1,CIP45CERT2,CIP45ASSOC,CIP45CERT4,CIP45BACHL,CIP46CERT1,CIP46CERT2,CIP46ASSOC,CIP46CERT4,CIP46BACHL,CIP47CERT1,CIP47CERT2,CIP47ASSOC,CIP47CERT4,CIP47BACHL,CIP48CERT1,CIP48CERT2,CIP48ASSOC,CIP48CERT4,CIP48BACHL,CIP49CERT1,CIP49CERT2,CIP49ASSOC,CIP49CERT4,CIP49BACHL,CIP50CERT1,CIP50CERT2,CIP50ASSOC,CIP50CERT4,CIP50BACHL,CIP51CERT1,CIP51CERT2,CIP51ASSOC,CIP51CERT4,CIP51BACHL,CIP52CERT1,CIP52CERT2,CIP52ASSOC,CIP52CERT4,CIP52BACHL,CIP54CERT1,CIP54CERT2,CIP54ASSOC,CIP54CERT4,CIP54BACHL,UGDS,UG,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,UGDS_WHITENH,UGDS_BLACKNH,UGDS_API,UGDS_AIANOLD,UGDS_HISPOLD,UG_NRA,UG_UNKN,UG_WHITENH,UG_BLACKNH,UG_API,UG_AIANOLD,UG_HISPOLD,PPTUG_EF,PPTUG_EF2,NPT4_PUB,NPT4_PRIV,NPT4_PROG,NPT4_OTHER,NPT41_PUB,NPT42_PUB,NPT43_PUB,NPT44_PUB,NPT45_PUB,NPT41_PRIV,NPT42_PRIV,NPT43_PRIV,NPT44_PRIV,NPT45_PRIV,NPT41_PROG,NPT42_PROG,NPT43_PROG,NPT44_PROG,NPT45_PROG,NPT41_OTHER,NPT42_OTHER,NPT43_OTHER,NPT44_OTHER,NPT45_OTHER,NPT4_048_PUB,NPT4_048_PRIV,NPT4_048_PROG,NPT4_048_OTHER,NPT4_3075_PUB,NPT4_3075_PRIV,NPT4_75UP_PUB,NPT4_75UP_PRIV,NPT4_3075_PROG,NPT4_3075_OTHER,NPT4_75UP_PROG,NPT4_75UP_OTHER,NUM4_PUB,NUM4_PRIV,NUM4_PROG,NUM4_OTHER,NUM41_PUB,NUM42_PUB,NUM43_PUB,NUM44_PUB,NUM45_PUB,NUM41_PRIV,NUM42_PRIV,NUM43_PRIV,NUM44_PRIV,NUM45_PRIV,NUM41_PROG,NUM42_PROG,NUM43_PROG,NUM44_PROG,NUM45_PROG,NUM41_OTHER,NUM42_OTHER,NUM43_OTHER,NUM44_OTHER,NUM45_OTHER,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,PFTFTUG1_EF,RET_FT4,RET_FTL4,RET_PT4,RET_PTL4,UG25ABV,AGE_ENTRY_SQ,AGEGE24,LNFAMINC,LNFAMINC_IND,PCT_WHITE,PCT_BLACK,PCT_ASIAN,PCT_HISPANIC,PCT_BA,PCT_GRAD_PROF,PCT_BORN_US,MEDIAN_HH_INC,POVERTY_RATE,UNEMP_RATE,LN_MEDIAN_HH_INC,FSEND_COUNT,FSEND_1,FSEND_2,FSEND_3,FSEND_4,FSEND_5,UGDS_MEN,UGDS_WOMEN,D_PCTPELL_PCTFLOAN,UGNONDS,GRADS,RET_FT4_POOLED,RET_FTL4_POOLED,RET_PT4_POOLED,RET_PTL4_POOLED,RET_FT_DEN4

In [17]:
df_pub_state = df_pub.merge(income_dictionary['2013_14'], left_on='STABBR', right_on='Code')
df_pub_state = df_pub_state.drop(['STABBR', 'Code'], axis=1)

In [18]:
df_priv_state = df_priv.merge(income_dictionary['2013_14'], left_on='STABBR', right_on='Code')
df_priv_state = df_priv_state.drop(['STABBR', 'Code'], axis=1)

In [19]:
df_priv_profit_state = df_priv_profit.merge(income_dictionary['2013_14'], left_on='STABBR', right_on='Code')
df_priv_profit_state = df_priv_profit_state.drop(['STABBR', 'Code'], axis=1)

## Random forest test with additional feature

In [20]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

In [21]:
def fitRandomForestRegressor(df):
    train_features = list(df.columns)
    train_features.remove('debt_to_income')
    
    # Instantiate model with 1000 decision trees
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, oob_score=True)

    rf.fit(df[train_features], df['debt_to_income'])
    
    return rf


In [22]:
def getImportantFeatures(rf, df):
    important_features_dict = {}
    for x,i in enumerate(rf.feature_importances_):
        important_features_dict[x]=i


    important_features_list = sorted(important_features_dict,
                                     key=important_features_dict.get,
                                     reverse=True)
    
    top_features = df.columns[important_features_list]
    
    return top_features[:20]

In [23]:
%%time
print('Important Features for Public Colleges/Universities: ')
rf_pub = fitRandomForestRegressor(df_pub_state)
top_features_pub = getImportantFeatures(rf_pub, df_pub_state)
print(top_features_pub)

Important Features for Public Colleges/Universities: 
Index(['SAT_AVG_ALL', 'RET_FT4', 'ADM_RATE_ALL', 'UGDS_BLACK', 'PPTUG_EF',
       'TUITIONFEE_IN', 'NPT4_PUB', 'PCIP50', 'COSTT4_A', 'UGDS_WHITE',
       'UGDS_ASIAN', 'UG25ABV', 'UGDS_HISP', 'NPT45_PUB', 'debt_to_income',
       'TUITIONFEE_OUT', 'UGDS_AIAN', 'UGDS_2MOR', 'NPT4_3075_PUB', 'PCIP14'],
      dtype='object')
CPU times: user 1min 21s, sys: 522 ms, total: 1min 22s
Wall time: 1min 22s


In [24]:
%%time
print('Important Features for Private Colleges/Universities: ')
rf_priv = fitRandomForestRegressor(df_priv_state)
top_features_priv = getImportantFeatures(rf_priv, df_priv_state)
print(top_features_priv)


Important Features for Private Colleges/Universities: 
Index(['PCIP50', 'TUITIONFEE_IN', 'ADM_RATE_ALL', 'PCIP23', 'RET_FT4',
       'PPTUG_EF', 'GRADS', 'UGDS_BLACK', 'NPT42_PRIV', 'NPT4_3075_PRIV',
       'PFTFTUG1_EF', 'UG25ABV', 'UGDS_WHITE', 'debt_to_income', 'PCIP52',
       'UGDS_HISP', 'SAT_AVG_ALL', 'UGDS', 'NPT41_PRIV', 'PCIP51'],
      dtype='object')
CPU times: user 1min 7s, sys: 308 ms, total: 1min 7s
Wall time: 1min 7s


In [25]:
%%time
print('Important Features for Private for Profit Colleges/Universities: ')
rf_priv_profit = fitRandomForestRegressor(df_priv_profit_state)
top_features_priv_profit = getImportantFeatures(rf_priv_profit, df_priv_profit_state)
print(top_features_priv_profit)


Important Features for Private for Profit Colleges/Universities: 
Index(['TUITIONFEE_IN', 'PCIP50', 'PPTUG_EF', 'GRADS', 'CIPTFBS1',
       'UGDS_WOMEN', 'PFTFTUG1_EF', 'UG25ABV', 'debt_to_income', 'UGDS_WHITE',
       'RET_FTL4', 'TUITIONFEE_OUT', 'UGDS_ASIAN', 'TUITIONFEE_PROG',
       'UGDS_HISP', 'UGDS_BLACK', 'UGDS', 'NUM4_PRIV', 'NUM41_PRIV',
       'MTHCMP1'],
      dtype='object')
CPU times: user 1min 47s, sys: 867 ms, total: 1min 47s
Wall time: 1min 59s


### Get OOB Score

In [26]:
print(rf_pub.oob_score_)
print(rf_priv.oob_score_)
print(rf_priv_profit.oob_score_)


0.75203317614994
0.5628389192086147
0.4786211753075742
